In [8]:
from urllib import parse

# Arguments

※ 네이버 뉴스는 한번에 4000개밖에 크롤링할 수 없음 --> 날짜를 잘게 쪼개어 수집  
※ 웹 크롤러는 네이버 뉴스의 서버에 동일한 요청을 반복적으로 보내기 때문에 바이러스로 의심받을 수 있음 --> 내 컴퓨터의 정보를 headers로 알려줌으로써 이상한 사람이 아니라는 것을 피력

In [1]:
url_base = 'https://search.naver.com/search.naver?where=news&query={}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={}&de={}&start={}'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"}

In [75]:
QUERY = '푸바오'
DATE_START = '2023.07.01'
DATE_END = '2023.07.01'
URL_START_IDX = '1'

url_list_page = url_base.format(parse.quote(QUERY), DATE_START, DATE_END, URL_START_IDX)

In [76]:
url_list_page

'https://search.naver.com/search.naver?where=news&query=%ED%91%B8%EB%B0%94%EC%98%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2023.07.01&de=2023.07.01&start=1'

# Parse List Page

In [77]:
import time
from urllib import request
from bs4 import BeautifulSoup

In [78]:
req = request.Request(url=url_list_page, headers=headers)

In [79]:
html = request.urlopen(req).read()

In [80]:
soup = BeautifulSoup(html, 'lxml')

In [81]:
url_list = []
for item in soup.find_all('a', class_='info'):
    if '네이버뉴스' in item:
        url_list.append(item.get('href'))
        
print(url_list)

['https://n.news.naver.com/mnews/article/666/0000015071?sid=101', 'https://n.news.naver.com/mnews/article/023/0003773051?sid=102', 'https://n.news.naver.com/mnews/article/366/0000913463?sid=105']


In [82]:
url_list = [url.get('href') for url in soup.find_all('a', class_='info') if '네이버뉴스' in url]

In [83]:
url_list

['https://n.news.naver.com/mnews/article/666/0000015071?sid=101',
 'https://n.news.naver.com/mnews/article/023/0003773051?sid=102',
 'https://n.news.naver.com/mnews/article/366/0000913463?sid=105']

# Parse Article Page

In [84]:
req = request.Request(url=url_list[0], headers=headers)
html = request.urlopen(req).read()
soup = BeautifulSoup(html, 'lxml')

In [93]:
for item in soup.find_all('h2', id='title_area'):
    print(item.get_text())

"판다 사육하고 50만원"…여름방학 이색알바 '눈길'


In [97]:
soup.find_all('span', class_='media_end_head_info_datestamp_time _ARTICLE_DATE_TIME')

[<span class="media_end_head_info_datestamp_time _ARTICLE_DATE_TIME" data-date-time="2023-07-01 10:02:01">2023.07.01. 오전 10:02</span>]

In [99]:
soup.find('span', class_='media_end_head_info_datestamp_time _ARTICLE_DATE_TIME').get_text()

'2023.07.01. 오전 10:02'

In [104]:
soup.find('div', id='dic_area').get_text()

"\n \n\n\n\n    당근마켓과 에버랜드가 함께 진행하는 '푸바오 매니저 알바'. 당근마켓 캡처      코로나19\u2008이후\u2008일자리\u2008구조\u2008변화와\u2008시급\u2008인상\u2008등으로\u2008알바\u2008자리가\u2008줄어드는\u2008가운데\u2008여름방학을\u2008맞은\u2008대학생들을\u2008사로\u2008잡는\u2008이색\u2008알바들이\u2008눈길을\u2008끌고\u2008있다.   먼저,\u2008당근마켓과\u2008에버랜드는\u2008지역\u2008기반\u2008구인·구직\u2008서비스\u2008‘당근알바’를\u2008통해\u2008색다른\u2008놀이공원\u2008알바\u2008체험을\u2008제공하는\u2008‘올여름 한량 알바’\u2008이벤트를\u2008진행하고\u2008있다.\u2008총\u2008200명(100명 추첨, 1인 동행)에게\u2008에버랜드에서\u2008신나는\u2008하루를\u2008보내는\u2008임무가\u2008부여되는\u2008체험형\u2008이벤트로\u2008이달\u20082일까지\u2008신청을\u2008받고\u2008있다.   그중에서도\u2008판다\u2008‘푸바오’의\u2008일일\u2008매니저를\u2008체험하는\u2008알바가\u2008청년층의\u2008큰\u2008인기를\u2008끌고\u2008있다.\u2008알바\u2008지원서에\u2008푸바오에\u2008대한\u2008애정을\u2008담아\u2008올리면,\u2008이달\u200810일부터\u200814일\u2008사이에\u2008푸바오\u2008사육사와\u2008함께\u2008체험\u2008프로그램에\u2008참여할\u2008수\u2008있다.\u2008총\u20083명을\u2008선발해\u2008당근머니\u200850만원을\u2008지급한다.   일일\u2008매니저에\u2008지원했다는\u2008대학생\u2008김현영씨(25·가명)는\u2008“사육사라는 직업에도\u2008관심

In [105]:
import re

In [106]:
text = soup.find('div', id='dic_area').get_text()
re.sub('\u2008', ' ', text)

"\n \n\n\n\n    당근마켓과 에버랜드가 함께 진행하는 '푸바오 매니저 알바'. 당근마켓 캡처      코로나19 이후 일자리 구조 변화와 시급 인상 등으로 알바 자리가 줄어드는 가운데 여름방학을 맞은 대학생들을 사로 잡는 이색 알바들이 눈길을 끌고 있다.   먼저, 당근마켓과 에버랜드는 지역 기반 구인·구직 서비스 ‘당근알바’를 통해 색다른 놀이공원 알바 체험을 제공하는 ‘올여름 한량 알바’ 이벤트를 진행하고 있다. 총 200명(100명 추첨, 1인 동행)에게 에버랜드에서 신나는 하루를 보내는 임무가 부여되는 체험형 이벤트로 이달 2일까지 신청을 받고 있다.   그중에서도 판다 ‘푸바오’의 일일 매니저를 체험하는 알바가 청년층의 큰 인기를 끌고 있다. 알바 지원서에 푸바오에 대한 애정을 담아 올리면, 이달 10일부터 14일 사이에 푸바오 사육사와 함께 체험 프로그램에 참여할 수 있다. 총 3명을 선발해 당근머니 50만원을 지급한다.   일일 매니저에 지원했다는 대학생 김현영씨(25·가명)는 “사육사라는 직업에도 관심이 많아 이번 알바에 지원했는데, 판다를 포함해 다양한 동물을 좋아하는 사람들에겐 매력적인 알바 중 하나인 것 같다”고 기대감을 드러냈다.   또 벤츠 등 수입 자동차 전시장의 ‘리셉션 알바’도 청년층 관심이 많은 이색 알바 중 하나다. 리셉션에서 차량 상담 등을 위해 전시장에 오는 손님들을 안내하는 업무로, 서비스직을 꿈꾸는 대학생들이 주로 찾는다.   최근에는 반려동물 문화의 확산에 따라 ‘애견 유치원’ 알바도 이색 알바로 주목받고 있다. 유치원에 ‘등원’한 반려동물의 위생관리부터 등·하원까지 전반적인 관리에 집중하는 업무로, 강아지나 고양이 등 반려동물에 관심이 많은 청년층에게 인기가 높다. 시급 역시 1만원 대로 적지 않은 수준이다.   전문가들은 청년층이 이색 알바에 집중하는 현상을 ‘재미를 추구하는 또래 문화’의 영향 중 하나라고 평가했다.   이은희 인하대 소비자학과 교수는 “청년층은 돈을 많이 버는 것도 중요하지만, 어떻게 

In [133]:
import numpy as np

In [137]:
np.

AttributeError: module 'numpy' has no attribute 'nrand'

In [119]:
naver = {}
for idx, url in enumerate(url_list):
    req = request.Request(url=url, headers=headers)
    html = request.urlopen(req).read()
    soup = BeautifulSoup(html, 'lxml')
    
    title = soup.find('h2', id='title_area').get_text()
    date = soup.find('span', class_='media_end_head_info_datestamp_time _ARTICLE_DATE_TIME').get_text()
    content = re.sub('\u2008', ' ', soup.find('div', id='dic_area').get_text())
    
    article_key = '{:04d}'.format(idx)
    naver[article_key] = {
        'title': title,
        'date': date,
        'content': content
    }
    
    time.sleep(3)

In [122]:
naver['0001']['date']

'2023.07.01. 오전 3:05'

In [123]:
naver['0002']['title']

'[애니멀리포트] 비쩍 마른 부경동물원 사자, 7년 만에 케이지 벗어나 흙 밟는다'

In [124]:
for article_key in naver.keys():
    print(naver[article_key]['title'])

"판다 사육하고 50만원"…여름방학 이색알바 '눈길'
섬마을 작은 학교 졸업 사진 남겨주러… 카메라 들고 배 타는 사진사들
[애니멀리포트] 비쩍 마른 부경동물원 사자, 7년 만에 케이지 벗어나 흙 밟는다


# Export Articles

In [125]:
import os
import json

In [126]:
fdir = 'C:\Google Drive/playground/23_workspace/bug/data/naver_news'
fname = 'naver.json'
fpath = os.path.join(fdir, fname)

with open(fpath, 'w', encoding='utf-8') as f:
    json.dump(naver, f)

In [127]:
with open(fpath, 'r', encoding='utf-8') as f:
    new_naver = json.load(f)

In [132]:
fdir = 'C:\Google Drive/playground/23_workspace/bug/data/naver_news'
for article_key, article in naver.items():
    fname = 'article_{}.json'.format(article_key)
    fpath_article = os.path.join(fdir, fname)
    with open(fpath_article, 'w', encoding='utf-8') as f:
        json.dump(article, f)